In [40]:
from openalex_matching import person_match
from collections import defaultdict
from openalex_matching import csv_handler


# OpenAlex API Search Strategy

# Strategy: 
1. Find the OpenAlex Institution ID:
    - First, search for the OpenAlex ID of the desired institution using the institution's name.

2. Search for Authors by Name and Institution:
    - Next, search for authors by their display name, filtering results by the institution's OpenAlex ID. If no results are found:
        - Search through nick names associated with author's first name
        - If the input name consists of two capital letters (indicating initials), attempt an initial-based search.
        - If that still returns no results, try searching by first initial and last name.
    - The search will return one of three enumerated types:
        - EXACT_NAME: Indicates an exact match with the full name.
        - FIRST_MIDDLE_INITIAL: Indicates a match using first and middle initials.
        - FIRST_INITIAL: Indicates a match using only the first initial.
    - The filtering method will adjust based on the type of search used.
3. Filter and Select the Best Matched Author:
    - Depending on the type of search used, filter the list of matched OpenAlex IDs:
        - Exact Name Search (EXACT_NAME): Use fuzzy string matching to eliminate names that deviate too much from the inputted author name. Select the author with the highest fuzzy match score. In case of a tie in fuzzy match scores, choose the author with the higher citation count.
        - Initial Search (FIRST_MIDDLE_INITIAL or FIRST_INITIAL): Ensure that the first initial of the inputted author name matches the first initial of the returned author names. Filter out any mismatches. Then, select the most highly cited author.
        - For searches using only one initial (FIRST_INITIAL), apply a stricter fuzzy matching threshold since these searches are more prone to errors. In case of a tie, select the author with more citations.


In [21]:
#Step One
university_name = 'University of Virginia'
university_id = person_match.institution_id_openalex(university_name)
print(university_id)

I51556381


In [26]:
#Step Two
person_name = "Ray Balkrishnan"
list_of_ids, type_search_conducted, match_person_name = person_match.list_person_ids_openalex(person_name, university_id)
print(list_of_ids)




{'raymond', 'Ray'}
[]
[]
['A5111842581', 'A5015605411', 'A5029765901']


In [27]:
#Step Three
openalex_id = person_match.choose_person(list_of_ids, match_person_name, university_id, type_search_conducted)
print(openalex_id)

A5029765901


## Example inputting and outputting CSV files

In [41]:
university_name='university of virginia'
university_id = person_match.institution_id_openalex(university_name)
fileName = 'OpenAlex Names.csv'
namesArray = csv_handler.name_csv_reader(fileName) # Reading all names from CSV file, appending to array
IDArray = [] # Array that will contain ids of corresponding authors
dataDict = defaultdict(list) # Dictionary that will be written to output CSV file

#Iterating through names array, finding corresponding OpenAlex ID for each author
for name in namesArray: 
    person_ids, type_search_conducted, match_person_name = person_match.list_person_ids_openalex(name, university_id)
    selectID = person_match.choose_person(person_ids, name, university_id, type_search_conducted)
    IDArray.append(selectID)

dataDict["Names"] = namesArray #Creating Names column for CSV file
dataDict["OpenAlexID"] = IDArray #Creating corresonding IDs column for CSV file
csv_handler.name_csv_writer("output.csv", dataDict)

{'gabby', 'ella', 'Gabrielle'}
{'Nafisa'}
[]
A5016085568 has not met string matching threshold: not a valid name
{'Sonia'}
{'Negin'}
{'Harsh'}
{'liz', 'beth', 'bessie', 'betty', 'eliza', 'lizzy', 'lisa', 'lizzie', 'liza', 'bess', 'libby', 'lib', 'Elizabeth', 'betsy'}
{'ant', 'Anthony', 'tony'}
[]
[]
[]
{'mamie', 'rosemary', 'polly', 'maureen', 'marilyn', 'mitzi', 'mae', 'demaris', 'mariah', 'marie', 'marietta', 'Mary', 'marsha', 'molly', 'demerias', 'marion'}
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


KeyboardInterrupt: 

In [2]:
def institution_id_openalex(university_name):

    """
    input: 
         uni_name: the name of the university or institution
    output:
         university_id: the openalex id of the university
    """
 
    # search an university based on its name; and only return the id, the display_name and works_count in the database
    url=f'https://api.openalex.org/institutions?search={university_name}&select=id,display_name,works_count'
        
    response = requests.get(url)
    
    if response.status_code!=200:
        print('The webpage cannot be fetched!')
        
    data = response.json()
    count=data['meta']['count']
    
    
    # check the number of items returned
    if count==0:
        print('no match for the institution!')
        
    elif count==1:
        university_id=data['results'][0]['id']
        
    else:
        # Find the ID of the university with the most publications   
        university_with_max = max(data['results'], key=lambda x: x["works_count"])
        university_id = university_with_max["id"]
        
    university_id=university_id.split('/')[-1]

    return university_id

In [3]:
def person_ids_openalex(person_name, institution_ids):

    """
    Input
      person_name: the name of a person
      institution_ids: a list of institution openalex ids the person afffiliated with
      
    Output
      person_ids: a list of openalex ids matched
      
    """

    ls_results=[]
    
    for institution_id in institution_ids:
        
        url=f'https://api.openalex.org/authors?search={person_name}&filter=affiliations.institution.country_code:us,affiliations.institution.type:education,affiliations.institution.id:{institution_id}&per-page=200'
        
        response = requests.get(url)
        
        if response.status_code!=200:
            print(f'The webpage cannot be fetched for {person_name}!')
            continue
            
        data = response.json()
        count=data['meta']['count']
        if count==0:
            print(f'no match for {person_name} for the institution')
            continue
            
        # Each person may be affiliated with multiple institutions. Select those whose half of the institutions are in the US
        select_results=[]
    
        for i in data['results']:
            country_codes=[j['institution']['country_code'] for j in i['affiliations']]
            item = 'US'
            count = country_codes.count(item)
            percent=count/len(country_codes)
    
            if percent>0.5:
                ls_results.append(i)
            else:
                continue
                
    person_ids=list(set([i['id'] for i in ls_results]))
    person_ids=[id.split('/')[-1] for id in person_ids]
    
    if len(person_ids)==0:
        print(f'no match for {person_name} finally!')
    
    return person_ids

In [14]:
university_name='University of Virginia'
university_id=institution_id_openalex(university_name)

institution_ids=[university_id]
person_name='Mark White'
person_ids=person_ids_openalex(person_name, institution_ids)

NameError: name 'institution_id_openalex' is not defined

In [15]:
person_ids

NameError: name 'person_ids' is not defined

In [6]:
def select_person(person_name, person_ids):

    """
    If one person has multiple records after using the 'person_ids_openalex' function, then use the person_name as an exact match. If a president's name is among the display_name_alternatives, select the person_id.

    input
      person_ids: a list of openalex person ids returned by the 'person_ids_openalex' function;

    output
      matched_ids: a list of ids that can be matched by the exact name match;
    
    """
    
    matched_ids=[]
    for person_id in tqdm(person_ids):
        
        url=f'https://api.openalex.org/authors/{person_id}'
        response = requests.get(url)
                
        if response.status_code!=200:
            print(f'The webpage cannot be fetched for {person_id}!')
            continue
        
        data = response.json()
        
        # check if the person_name is among the alternative
        alts=[i.lower() for i in data['display_name_alternatives']]

        if person_name in alts:
            matched_ids.append(person_id)
    
    return matched_ids

In [7]:
if len(person_ids)>1:
    matched_ids=select_person(person_name, person_ids)

In [17]:
def exact_name_match(person_name):

    """
    This function obtains the openalex ID for a person based solely on the person_name. In this case, the names should exactly appear in the list of alternative names, and over half of the person's affiliations are in the US.

    Input:
      person_name: the name of a person
      
    Output:
      author_id: the openalex id for the person
    
    """

    url=f'https://api.openalex.org/authors?search={person_name}&filter=affiliations.institution.country_code:us,affiliations.institution.type:education&per-page=200'
            
    response = requests.get(url)
    
    if response.status_code!=200:
        print(f'The webpage cannot be fetched for {person_name}!')
        
    data = response.json()
    count=data['meta']['count']
    if count==0:
        print(f'no match for {person_name} for the institution')
    
  
    ids=[i['id'] for i in data['results']]
    alts=[i['display_name_alternatives'] for i in data['results']]
    
    country_per_person=[]
    for i in data['results']:
        country_codes=[j['institution']['country_code'] for j in i['affiliations']]
        item = 'US'
        count = country_codes.count(item)
        percent=count/len(country_codes)
        country_per_person.append(percent)
    
    selected_pairs = [(id,x, y) for id,x, y in zip(ids,country_per_person, alts) if x > 0.5 and person_name in y]
    author_id=selected_pairs[0][0].split('/')[-1]
    
    return author_id

In [19]:
exact_name_match(person_name)

NameError: name 'person_name' is not defined

In [10]:
def name_csv_reader(fileName):
    df = pd.read_csv(fileName)
     # Check if 'Names' column exists
    if 'Names' in df.columns:
        namesArray = df['Names'].to_numpy()
    else:
        # Handle the case where 'Names' column is missing
        print("Error: 'Names' column not found in the CSV file.")
        namesArray = []

    return namesArray

In [11]:
fileName = 'OpenAlex Names.csv'
namesArray = name_csv_reader(fileName)
print(namesArray)

['Gabrielle Adams' 'Nafisa Ahmed' 'Sonia Alconini' 'Negin Alemazkoor'
 'Harsh Anand' 'Elizabeth Andrews' 'Anthony Artuso' 'Mary Asare-Ado'
 'Ehsan Baharlou' 'Teagan Baiotto' 'Cora Baird' 'Prasanna Balachandran'
 'Ray Balkrishnan' 'Lawrence Band' 'Ellen Bassett' 'Tim Beatley'
 'Jeffrey Bennett' 'Peter Berg' 'Emily Bernhardt' 'Alice Besterman'
 'Lori Bird' 'Shannon Blevins' 'Swatah Borkotoky' 'Hanne Borstlap'
 'Allison Bradshaw' 'Jeanine Braithwaite' 'JD Brown' 'Kelly Bulin'
 'Mark Buntaine' 'Matthew Burtner' 'Dawn Byrd' 'Liheng Cai'
 'Brad Campbell' 'Jonathan Cannon' 'Brad Cantrell' 'Kerrie Carfagno'
 'Ann Carlton' 'Kathy Carmody' 'Sergio Casas' 'Max Castorani' 'Dong Chen'
 'Donna Chen' 'Leena Cho' 'Michele Claibourn' 'Andres Clarens'
 'Tanya Cobb' 'Ronald Cohen' 'Jonathan Colmer' 'Lisa Colosi'
 'Josh Colston' 'Ben Converse' 'Nina Copeland' 'Sheila Crane'
 'Phoebe Crisman' 'Teresa Culver' "Paolo D'Odorico" 'Robert Davis'
 'Brian Davis' 'Peter Debaere' 'Cahterine Debbas' 'Pam DeGuzman'
 

In [12]:
def dict_csv_writer(fileName, dataToWrite):
    df = pd.DataFrame.from_dict(dataToWrite)
    df.to_csv(fileName, index=False)
    print("data appended to csv")

In [13]:
def nameParser(person_name):
     
     """
    Input
      person_name: the name of a person
      
    Output
      first_name: person's first name
      last_name: person's last name
      
    """
     name_split = person_name.strip().split()
     if (len(name_split) < 2):
          return "Invalid Name, need both first and last"
     first_name = name_split[0]
     last_name = name_split[-1]
    
     return first_name, last_name
    

In [14]:
person_name = "Eli Stine"
print(nameParser(person_name))
institution_ids = [university_id]

('Eli', 'Stine')


In [263]:
class SearchType(Enum):
    EXACT_NAME = 0
    FIRST_MIDDLE_INITIAL = 1
    FIRST_INITIAL = 2

In [272]:
def list_person_ids_openalex(person_name, university_id):
    """
    Input
      person_name: the name of a person
      
    Output
      person_ids: a list of openalex ids matched
    """
    nn = NickNamer()
    first, last = nameParser(person_name)
    totalNames = {first} | set(nn.nicknames_of(first)) | set(nn.canonicals_of(first))
    typeOfSearchConducted = SearchType.EXACT_NAME
    print(totalNames)
    
    ids = []
    bestNameMatch = first
    
    def search_with_name(name):
        url = f'https://api.openalex.org/authors?filter=affiliations.institution.id:{university_id}&search={name}%20{last}'
        trycnt = 3
        while trycnt > 0:
            try:
                response = requests.get(url)
                if response.status_code != 200:
                    print(f'The webpage cannot be fetched for {name} {last}!')
                    break
                
                data = response.json()
                
                idUrl = [result['id'] for result in data['results']]
                return [url.split('/')[-1] for url in idUrl]
                
            except (ConnectionError, ConnectionResetError) as ex:
                if trycnt <= 1:
                    print(f"Failed to retrieve: {url}\n{str(ex)}")  # Done retrying
                else:
                    print(f"Retrying... ({3 - trycnt + 1}/3)")
                    trycnt -= 1  # Decrement retry counter
                    time.sleep(0.5)  # Wait half a second before retrying
        return []

    ids.extend(search_with_name(first))
    # First, search with all names in totalNames
    if not ids:
        for firstName in totalNames:
            ids.extend(search_with_name(firstName))
            print(ids)
            if ids:
                bestNameMatch = firstName
                break
        person_name = f"{bestNameMatch} {last}"
    # If no results found, search with initial
    if not ids:
        if len(first) == 2:
            initials = first[0] + ".%20" + first[1] + "."
            ids.extend(search_with_name(initials))
            typeOfSearchConducted = SearchType.FIRST_MIDDLE_INITIAL
        else:
            initial = first[0] + "."
            ids.extend(search_with_name(initial))
            typeOfSearchConducted = SearchType.FIRST_INITIAL
    return ids, typeOfSearchConducted, person_name

In [124]:
institution_ids = [university_id]
person_ids = list_person_ids_openalex(person_name, university_id)
print(person_ids)

{'fred', 'willie', 'robert', 'william', 'will', 'willis', 'Bill', 'billy'}
https://api.openalex.org/authors?filter=affiliations.institution.id:I51556381&search=fred%20Shobe
https://api.openalex.org/authors?filter=affiliations.institution.id:I51556381&search=willie%20Shobe
https://api.openalex.org/authors?filter=affiliations.institution.id:I51556381&search=robert%20Shobe
https://api.openalex.org/authors?filter=affiliations.institution.id:I51556381&search=william%20Shobe
https://api.openalex.org/authors?filter=affiliations.institution.id:I51556381&search=will%20Shobe
https://api.openalex.org/authors?filter=affiliations.institution.id:I51556381&search=willis%20Shobe
https://api.openalex.org/authors?filter=affiliations.institution.id:I51556381&search=Bill%20Shobe
https://api.openalex.org/authors?filter=affiliations.institution.id:I51556381&search=billy%20Shobe
https://api.openalex.org/authors?filter=affiliations.institution.id:I51556381&search=B.%20Shobe
['A5025016843']


In [94]:
def topic_id_openAlex(topicName):

    url = f"https://api.openalex.org/topics?search={topicName}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f'The webpage cannot be fetched for {topicName}!')

    data = response.json()
    if data['meta']['count'] == 0:
        return f"{topicName} is an invalid topic name"
    else:
        topicID = data['results'][0]['id']
        topicID = topicID.split("/")[-1]
    return topicID

In [222]:
topicID = topic_id_openAlex("American Political Thought and History")
print(topicID)

T13445


In [229]:
def list_person_ids_openalex_by_topic(person_name, university_id, topicID):
    """
    Input
      person_name: the name of a person
      
    Output
      person_ids: a list of openalex ids matched
    """
    nn = NickNamer()
    first, last = nameParser(person_name)
    totalNames = {first} | set(nn.nicknames_of(first)) | set(nn.canonicals_of(first))
    print(totalNames)
    
    ids = []
    
    def search_with_name(name):
        url = f'https://api.openalex.org/authors?filter=affiliations.institution.id:{university_id}&search={name}%20{last}'
        trycnt = 3
        while trycnt > 0:
            try:
                response = requests.get(url)
                if response.status_code != 200:
                    print(f'The webpage cannot be fetched for {name} {last}!')
                    break
                
                data = response.json()
                
                filtered_ids = []
                for result in data['results']:
                    author_concepts = [concept['id'].split('/')[-1] for concept in result.get('topics', [])]
                    print(author_concepts)
                    if topicID in author_concepts:
                        filtered_ids.append(result['id'].split('/')[-1])  # Extract OpenAlex ID
                
                return filtered_ids
                
            except (ConnectionError, ConnectionResetError) as ex:
                if trycnt <= 1:
                    print(f"Failed to retrieve: {url}\n{str(ex)}")  # Done retrying
                else:
                    print(f"Retrying... ({3 - trycnt + 1}/3)")
                    trycnt -= 1  # Decrement retry counter
                    time.sleep(0.5)  # Wait half a second before retrying
        return []

    # First, search with all names in totalNames
    for firstName in totalNames:
        ids.extend(search_with_name(firstName))
    
    # If no results found, search with initial
    if not ids:
        if len(first) == 2:
            initials = first[0] + ".%20" + first[1] + "."
            ids.extend(search_with_name(initials))
        else:
            initial = first[0] + "."
            ids.extend(search_with_name(initial))
    return ids

In [230]:
personIDS = list_person_ids_openalex_by_topic("Craig Volden", university_id, topicID)
print(personIDS)

{'Craig'}
https://api.openalex.org/authors?filter=affiliations.institution.id:I51556381&search=Craig%20Volden
['T10108', 'T12210', 'T12225', 'T11461', 'T10443', 'T10802', 'T11432', 'T11770', 'T13445', 'T11544', 'T10213', 'T13138', 'T12619', 'T12185', 'T10991', 'T13158', 'T11621', 'T14013', 'T10289', 'T11762', 'T10391', 'T14509', 'T13710', 'T12961', 'T11397']
['A5004704074']


In [267]:
def choose_person(person_ids, person_name, university_id, typeOfSearchConducted):
    '''
    
    Input
      person_name: the name of a person
      person_ids: assoicated ids with a person (can be found with person_ids_alexa)
      university_id: university id for wh oyo
      
    Output
      person_ids: a list of openalex ids matched


    '''
    filtered_persons_ids = []
    selectID = " "
    maxCiteCount = -1
    firstName, lastName = nameParser(person_name)
    maxNameSimilarity = 0
    highThres, lowThres = 76, 65
    for id in person_ids:
        url = f'https://api.openalex.org/people/{id}'
        trycnt = 3
        
        while trycnt > 0:
            try:
                response = requests.get(url)
                if response.status_code != 200:
                    print(f'The webpage cannot be fetched for {person_name}!')
                    break
                
                data = response.json()
                affiliations = data.get('affiliations', [])
                institution_ids = [aff['institution']['id'].split('/')[-1] for aff in affiliations]
                # Proceed only if the specified university_id is in the list of institution_ids
                if university_id not in institution_ids:
                    break
                
                filtered_persons_ids.append(id)

                apiFirstName, apiLastName = nameParser(data['display_name'])
                apiDisplayName = apiFirstName + " " + apiLastName
                name_similarity = fuzz.ratio(person_name, apiDisplayName)

                if (typeOfSearchConducted == SearchType.FIRST_INITIAL):
                    threshold = highThres
                elif (typeOfSearchConducted == SearchType.EXACT_NAME):
                    threshold = lowThres
                else:
                    threshold = 0
                
                if name_similarity >= threshold:
                    if (typeOfSearchConducted == SearchType.FIRST_INITIAL or typeOfSearchConducted == SearchType.FIRST_MIDDLE_INITIAL):
                        if (firstName[0].lower() != apiFirstName[0].lower()):
                            print(f"{id} has initial mismatch: not a valid name")
                            break
                    if name_similarity > maxNameSimilarity:
                        maxNameSimilarity = name_similarity
                        maxCiteCount = data['cited_by_count']
                        selectID = id
                    elif name_similarity == maxNameSimilarity:
                        if data['cited_by_count'] > maxCiteCount:
                            maxCiteCount = data['cited_by_count']
                            selectID = id
                else:
                    print(f'{id} has not met string matching threshold: not a valid name')
    
                trycnt = 0  # Success, exit retry loop

            except (ConnectionResetError, ConnectionError) as ex:
                if trycnt <= 1:
                    print(f"Failed to retrieve: {url}\n{str(ex)}")  # Done retrying
                else:
                    print(f"Retrying... ({3 - trycnt + 1}/3)")
                    trycnt -= 1  # Decrement retry counter
                    time.sleep(0.5)  # Wait half a second before retrying

    return selectID
   

In [21]:
university_name='university of virginia'
university_id=institution_id_openalex(university_name)
selectID = choose_person(person_ids, person_name, university_id)

In [19]:
def get_orcid_ID(id):

    url = f'https://api.openalex.org/people/{id}'
    trycnt = 3
    while trycnt > 0:
        try:
            response = requests.get(url)
            
            if response.status_code != 200:
                print(f'The webpage cannot be fetched for {id}!')
                return None
            
            data = response.json()
            orcid_url = data.get('orcid', None)
            
            if orcid_url:
                orcidID = orcid_url.split('/')[-1]
                trycnt = 0  # Success, exit retry loop
                return orcidID
            else:
                print(f'No ORCID ID found for {id}.')
                return None

        except (ConnectionResetError, ConnectionError) as ex:
            if trycnt <= 1:
                print(f"Failed to retrieve: {url}\n{str(ex)}")  # Done retrying
                return None
            else:
                print(f"Retrying... ({3 - trycnt + 1}/3)")
                trycnt -= 1  # Decrement retry counter
                time.sleep(0.5)  # Wait half a second before retrying


In [20]:
orcidID = get_orcid_ID(selectID)
print(orcidID)

0000-0002-5039-0065


In [15]:
def search_orcid_ID(id):

     url = f'https://api.openalex.org/authors?filter=orcid:{id}'
     response = requests.get(url)
     data = response.json()
      
     if len(data.get('results' , [])) == 0:
        print("No author found for orcid id")
        return None

     return data['results'][0]['display_name']

     
    

In [16]:
testOrcid = "0000-0002-7234-185"
print(search_orcid_ID(testOrcid))


No author found for orcid id
None


In [269]:
person_name = "MC Forelle"
# Define university_name and university_id first
university_name = 'university of virginia'
university_id = institution_id_openalex(university_name)
# Now use university_id in the function call
print(nameParser(person_name))
institution_ids = [university_id]
# Get person IDs and check if an initial search was performed.
person_ids, initialSearchConducted, person_name = list_person_ids_openalex(person_name, university_id)
print(person_ids)
print(person_name)
# Choose the best matching person based on additional criteria.
selectID = choose_person(person_ids, person_name, university_id, initialSearchConducted)
print(f"{person_name}'s OpenAlex ID is: {selectID}")
orcidID = get_orcid_ID(selectID)
print(f"{person_name}'s ORCID ID is: {orcidID}")

('MC', 'Forelle')
{'mack', 'mac', 'MC'}
https://api.openalex.org/authors?filter=affiliations.institution.id:I51556381&search=MC%20Forelle
['A5093880368']
MC Forelle
MC Forelle's OpenAlex ID is: A5093880368
No ORCID ID found for A5093880368.
MC Forelle's ORCID ID is: None


In [273]:
university_name='university of virginia'
university_id=institution_id_openalex(university_name)
fileName = 'OpenAlex Names.csv'
namesArray = name_csv_reader(fileName)
IDArray = []
orcidArray = []
dataDict = defaultdict(list)
for name in namesArray:
    person_ids, initialSearchConducted, person_name = list_person_ids_openalex(name, university_id)
    selectID = choose_person(person_ids, name, university_id, initialSearchConducted)
    orcidID = get_orcid_ID(selectID)
    IDArray.append(selectID)
    orcidArray.append(orcidID)
print(IDArray)
print(orcidArray)
dataDict["Names"] = namesArray
dataDict["OpenAlexID"] = IDArray
dataDict["OrcidID"] = orcidArray
dict_csv_writer("output.csv", dataDict)


   





{'ella', 'gabby', 'Gabrielle'}
{'Nafisa'}
[]
A5016085568 has not met string matching threshold: not a valid name
The webpage cannot be fetched for  !
{'Sonia'}
No ORCID ID found for A5022662610.
{'Negin'}
{'Harsh'}
{'lizzy', 'bess', 'eliza', 'liza', 'Elizabeth', 'bessie', 'liz', 'libby', 'lib', 'betsy', 'lisa', 'lizzie', 'beth', 'betty'}
{'Anthony', 'tony', 'ant'}
[]
[]
[]
The webpage cannot be fetched for  !
{'demaris', 'mamie', 'marietta', 'marilyn', 'marsha', 'demerias', 'marie', 'molly', 'mariah', 'maureen', 'polly', 'Mary', 'mae', 'rosemary', 'mitzi', 'marion'}
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
The webpage cannot be fetched for  !
{'Ehsan'}
{'Teagan'}
No ORCID ID found for A5046437867.
{'corinne', 'Cora'}
No ORCID ID found for A5104116424.
{'Prasanna'}
{'Ray', 'raymond'}
[]
[]
No ORCID ID found for A5029765901.
{'lonny', 'laurinda', 'Lawrence', 'larry', 'lorne', 'lawrie', 'lon', 'lorry'}
{'Ellen', 'helena', 'ellender', 'helene', 'ella', 'nellie', 'lena', 'nell', 'hel